In [1]:
import requests
from bs4 import BeautifulSoup
import re
import sys
from time import sleep

In [2]:
headers = {
    "user-agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.99 Safari/537.36"
}

parms = {
    "pagenum" : 1,
    "Depth" : 3111
}

In [3]:
url = "http://disability.seoul.go.kr/amenity/sightSeeingConven/sightSeeingConven.jsp"

In [4]:
def getUrl(url, parameter=None, num_retries=2, select=None):
    html = requests.get(url, params=parameter, headers=headers)
    
    if 500 <= html.status_code  < 600 and num_retries > 0:
        print(html.status_code, html.reason)
        return getUrl(url, params, num_retries=1)
        
    dom = BeautifulSoup(html.text, "lxml")
    links = dom.select(select)
    
    return links

In [5]:
select_css = { 'place': 'ul.travel-list2 > li > dl > dt > a ',
               'address' : 'ul.travel-list2 > li > dl > dd > p',
               'phone' : '.tel',
               'facility': 'ul.travel-list2 > li > dl > dd > ul.facil' }

In [6]:
place_Depth_code = {
    "korean" : 3111,
    "western" : 3211,
    "chinese"  : 3311,
    "japanese" : 3411,
    "flour_based" : 3511,
    "other" : 3611,
}

# 1. 한식 2. 양식 3. 중식 4. 일식 5. 분식 6. 기타  

In [7]:
korean = []
western = []
chinese = []
japanese = []
flour_based = []
other = []

In [10]:
# 1. 한식 2. 양식 3. 중식 4. 일식 5. 분식 6. 기타  
# type 구조 [{ }] 리스안에 딕셔너리 구조 
final_dict = {
'korean' : None ,
'western' : None ,
'chinese' : None ,
'japanese' : None ,
'flour_based' : None ,
'other' : None,

}

In [11]:
for place in place_Depth_code:
    place_code = place_Depth_code[place]
    parms['Depth'] = place_code
    result = []  
    while True:
        for select in select_css:
            get_info = getUrl(url, parameter = parms, select = select_css[select])
            # 장소, 장소_사이트, 주소, 전화번호, 편의시설
            if 'place' is select:
                place_name = [info.text for info in get_info] # 장소
                place_href = [info['href'] for info in get_info] # 장소_사이트
            if 'address' is select:
                address_pre = [re.sub(u'\xa0 \r\n\t*',"",info.text) for info in get_info] # 주소
                address =[re.sub(u'주소 :  ',"",info) for info in address_pre]
#             if 'phone' is select:
#                 phone = [info.text for info in get_info] # 전화번호
#             if 'facility' is select:
#                 facility_pre_select = [info for info in get_info] # 편의시설
                
        if get_info: 
            for info_num in range(0, len(place_name)):
#                 facility_select =  facility_pre_select[info_num].select('li > img')
#                 facility_select = [row_facility['alt'] for row_facility in facility_select]
#                 print(facility_select)
                info_dict = {'place_name' : place_name[info_num],
                             'place_href' : 'http://disability.seoul.go.kr'+place_href[info_num],
                             'place_address' : address[info_num],
#                              'place_facility' : facility_select
                             }  #     #
                result.append(info_dict)
            print(info_dict)
            parms['pagenum'] = parms['pagenum'] + 1
            # pagenum ->for next page
        else:
            parms['pagenum'] = 1
            break
    final_dict[place] = result

{'place_name': '가야옥', 'place_href': 'http://disability.seoul.go.kr/amenity/sightSeeingConven/sightSeeingConvenDetail.jsp?Depth=3111&seq=1337', 'place_address': '서울특별시 도봉구 도당로   2'}
{'place_name': '갈비예찬', 'place_href': 'http://disability.seoul.go.kr/amenity/sightSeeingConven/sightSeeingConvenDetail.jsp?Depth=3111&seq=1316', 'place_address': '서울특별시 도봉구 노해로 259'}
{'place_name': '개성면옥(함흥냉면)', 'place_href': 'http://disability.seoul.go.kr/amenity/sightSeeingConven/sightSeeingConvenDetail.jsp?Depth=3111&seq=987', 'place_address': '서울특별시 강서구 양천로 660'}
{'place_name': '고기굼터', 'place_href': 'http://disability.seoul.go.kr/amenity/sightSeeingConven/sightSeeingConvenDetail.jsp?Depth=3111&seq=990', 'place_address': '서울특별시 강서구 양천로 376'}
{'place_name': '교촌1991', 'place_href': 'http://disability.seoul.go.kr/amenity/sightSeeingConven/sightSeeingConvenDetail.jsp?Depth=3111&seq=1221', 'place_address': '서울특별시 서초구 강남대로69길  8'}
{'place_name': '근린생활시설(배밭고을)', 'place_href': 'http://disability.seoul.go.kr/amenit

{'place_name': '뚜레쥬르 <제과점>', 'place_href': 'http://disability.seoul.go.kr/amenity/sightSeeingConven/sightSeeingConvenDetail.jsp?Depth=3211&seq=1010', 'place_address': '서울특별시 도봉구 도봉로180길 6'}
{'place_name': '미스터피자 양재점(크리스탈빌딩)', 'place_href': 'http://disability.seoul.go.kr/amenity/sightSeeingConven/sightSeeingConvenDetail.jsp?Depth=3211&seq=984', 'place_address': '서울특별시 강남구 강남대로  242'}
{'place_name': '붓처스컷(SG다이힐빌딩)', 'place_href': 'http://disability.seoul.go.kr/amenity/sightSeeingConven/sightSeeingConvenDetail.jsp?Depth=3211&seq=1154', 'place_address': '서울특별시 강남구 압구정로60길 18'}
{'place_name': '빕스 문정점', 'place_href': 'http://disability.seoul.go.kr/amenity/sightSeeingConven/sightSeeingConvenDetail.jsp?Depth=3211&seq=1009', 'place_address': '서울특별시 송파구 새말로5길 31'}
{'place_name': '샤델리(한수빌딩)', 'place_href': 'http://disability.seoul.go.kr/amenity/sightSeeingConven/sightSeeingConvenDetail.jsp?Depth=3211&seq=1203', 'place_address': '서울특별시 강남구 언주로172길  62'}
{'place_name': '아웃백스테이크하우스 신대방점', 'place_hre

In [59]:
number_len =0 
for place in place_Depth_code:
    number_len += len(final_dict[place])
print(number_len)

414


In [13]:
facility_select =  "div.imgIcoBview" # 필요한 부분만 자르기

In [14]:
facility_available_name_select = "dt > a" # 시설물 설치된 이름
facility_is_available_select = "dt > span" # 시설물 이용가능 or 이용 불편 
facility_info_select = "ul.bulListDot > li" # 시설물 상세 정보 
# place_name  = "div.viewTitle p.sisulTitle" # 시설물 상세 정보 

In [17]:
final_dicts = {}
info_result = [] 
for place in place_Depth_code:

    for i, info in enumerate(final_dict[place]):
        info_result = [] 
        get_info = getUrl(info['place_href'], select = facility_select)
        facility_available_name= [info.text for info in get_info[0].select(facility_available_name_select)]
        facility_is_available = [info.text for info in get_info[0].select(facility_is_available_select)]
        facility_info = [info for info in get_info[0].select(facility_info_select)]
        facility_info = [info.__str__() for info in facility_info]
        facility_info = [ re.sub('^<li>|-|(<br/>)?</li>$' , '' , info).split('<br/>') for info in facility_info]
        
        for info_num in range(0, len(facility_available_name)):
            info_dict  = {'facility_available_name' : facility_available_name[info_num],
                     'facility_is_available' : facility_is_available[info_num],
                     'facility_info' : facility_info[info_num]
                     }
            info_result.append(info_dict)
        a = str(i)
        final_dict[place][i]['facility_detail_info'] = info_result
        info_result = None 
# print( final_dict[place][i]['facility_detail_info'])
#         print( final_dict[place][i]['facility_detail_info'])
#         print(final_dict[place][i]['facility_detail_info'])

[{'facility_available_name': '주출입구 접근로', 'facility_is_available': '이용가능', 'facility_info': ['상세정보 없음']}, {'facility_available_name': '장애인 전용 주차구역', 'facility_is_available': '이용가능', 'facility_info': ['총주차대수8', '전용주차대수  1']}, {'facility_available_name': '주출입구 높이차이 제거', 'facility_is_available': '이용불편', 'facility_info': ['경사로 1/12초과 설치', '유효폭 1.2m 이상']}]


In [18]:
import json
import urllib.request
import urllib.request
import datetime
import time
import json

In [19]:
def get_request_url(url):
    
    client_id="R63uNWNQiaTp1EeZl7bW"
    client_secret = "1Im1JohssC"
    
    req = urllib.request.Request(url)
    req.add_header("X-Naver-Client-Id", client_id)
    req.add_header("X-Naver-Client-Secret", client_secret)
    try: 
        response = urllib.request.urlopen(req)
        if response.getcode() == 200:
            print ("[%s] Url Request Success" % datetime.datetime.now())
            return response.read().decode('utf-8')
    except Exception as e:
        print(e)
        print("[%s] Error for URL : %s" % (datetime.datetime.now(), url))
        return None
    
def getGeoData(address):
    
    base = "https://openapi.naver.com/v1/map/geocode"
    node = ""
    parameters = "?query=%s" % urllib.parse.quote(address)
    url = base + node + parameters
    
    retData = get_request_url(url)
    
    if (retData == None):
        return None
    else:
        return json.loads(retData)

In [20]:
for place in place_Depth_code:
    info_result = [] 
    print(place)
    
    for info_num,info in enumerate(final_dict[place]):
        print(info['place_address'])
        jsonResult = getGeoData(info['place_address'])
        
        try:
            for item in jsonResult['result']['items']:
                    final_dict[place][info_num]['lat'] =  str(item['point']['y'])
                    final_dict[place][info_num]['lon'] =  str(item['point']['x'])
        except Exception as e:
            print("예외!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
            final_dict[place][info_num]['lat'] = "NaN"
            final_dict[place][info_num]['lon']=  "NaN"    
            print(e)

korean
서울특별시 강서구 공항대로  318
[2018-08-04 16:56:15.561365] Url Request Success
서울특별시 송파구 백제고분로 547
HTTP Error 404: Not Found
[2018-08-04 16:56:15.708972] Error for URL : https://openapi.naver.com/v1/map/geocode?query=%EC%84%9C%EC%9A%B8%ED%8A%B9%EB%B3%84%EC%8B%9C%20%EC%86%A1%ED%8C%8C%EA%B5%AC%20%EB%B0%B1%EC%A0%9C%EA%B3%A0%EB%B6%84%EB%A1%9C%20547
예외!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
'NoneType' object is not subscriptable
서울특별시 노원구 석계로 103
[2018-08-04 16:56:15.904449] Url Request Success
서울특별시 강남구 언주로153길  12
[2018-08-04 16:56:16.087961] Url Request Success
서울특별시 도봉구 도당로   2
[2018-08-04 16:56:16.259501] Url Request Success
서울특별시 영등포구 가마산로 330
[2018-08-04 16:56:16.499857] Url Request Success
서울특별시 노원구 덕릉로 692
[2018-08-04 16:56:16.657441] Url Request Success
서울특별시 서초구 강남대로  204
[2018-08-04 16:56:16.969604] Url Request Success
서울특별시 구로구 경인로 281
[2018-08-04 16:56:17.159097] Url Request Success
서울특별시 도봉구 노해로 259
[2018-08-04 16:56:17.523157] Url Request Success
서울특별시 강남구 언주로 855
[2018-08-04 16

[2018-08-04 16:56:33.866447] Url Request Success
서울특별시 성동구 마장로   317
[2018-08-04 16:56:34.031005] Url Request Success
서울특별시 강남구 도곡로63길  20
[2018-08-04 16:56:34.150688] Url Request Success
서울특별시 강서구 양천로 608
[2018-08-04 16:56:34.267373] Url Request Success
서울특별시 강남구 압구정로75길  6
[2018-08-04 16:56:34.431964] Url Request Success
서울특별시 용산구 회나무로  75
[2018-08-04 16:56:34.580539] Url Request Success
서울특별시 강동구 양재대로85길 16
[2018-08-04 16:56:34.698230] Url Request Success
서울특별시 강남구 도산대로 311
[2018-08-04 16:56:34.807928] Url Request Success
서울특별시 강남구 영동대로 725
[2018-08-04 16:56:34.949755] Url Request Success
서울특별시 송파구 문정로4길  14-38
HTTP Error 404: Not Found
[2018-08-04 16:56:35.078208] Error for URL : https://openapi.naver.com/v1/map/geocode?query=%EC%84%9C%EC%9A%B8%ED%8A%B9%EB%B3%84%EC%8B%9C%20%EC%86%A1%ED%8C%8C%EA%B5%AC%20%EB%AC%B8%EC%A0%95%EB%A1%9C4%EA%B8%B8%20%2014-38
예외!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
'NoneType' object is not subscriptable
서울특별시 중구 동호로14길 18
[2018-08-04 16:56:35.257726] Url R

[2018-08-04 16:56:49.975414] Url Request Success
서울특별시 강남구 도산대로45길 18
[2018-08-04 16:56:50.111057] Url Request Success
서울특별시 강남구 언주로168길  16
[2018-08-04 16:56:50.265639] Url Request Success
서울특별시 도봉구 시루봉로 68
[2018-08-04 16:56:50.394296] Url Request Success
서울특별시 도봉구 도봉로180길 6
[2018-08-04 16:56:50.532929] Url Request Success
서울특별시 은평구 응암로 229
[2018-08-04 16:56:50.683523] Url Request Success
서울특별시 강동구 양재대로 1651
[2018-08-04 16:56:50.813175] Url Request Success
서울특별시 서대문구 연세로 10
[2018-08-04 16:56:50.959784] Url Request Success
서울특별시 도봉구 노해로  285
[2018-08-04 16:56:51.113374] Url Request Success
서울특별시 강남구 강남대로  242
[2018-08-04 16:56:51.290217] Url Request Success
서울특별시 성북구 도봉로 1
[2018-08-04 16:56:51.427850] Url Request Success
서울특별시 강남구 도산대로  334
[2018-08-04 16:56:51.599391] Url Request Success
서울특별시 강남구 도산대로45길 18-2
[2018-08-04 16:56:51.745998] Url Request Success
서울특별시 용산구 이태원로 243
[2018-08-04 16:56:51.879652] Url Request Success
서울특별시 강남구 압구정로60길 18
[2018-08-04 16:56:52.044203] Url Reques

[2018-08-04 16:57:05.845325] Url Request Success
서울특별시 서초구 서초중앙로33길 8
[2018-08-04 16:57:05.984955] Url Request Success
서울특별시 성동구 고산자로26길 12
[2018-08-04 16:57:06.117601] Url Request Success
서울특별시 종로구 대학로10길 5
[2018-08-04 16:57:06.264462] Url Request Success
서울특별시 강서구 공항대로46길  90
[2018-08-04 16:57:06.401959] Url Request Success
서울특별시 은평구 연서로  165
[2018-08-04 16:57:06.527503] Url Request Success
서울특별시 마포구 어울마당로 147-1
[2018-08-04 16:57:06.666498] Url Request Success
서울특별시 강동구 강동대로53길 22
[2018-08-04 16:57:06.799775] Url Request Success
서울특별시 강북구 도봉로10가길 12
[2018-08-04 16:57:06.927436] Url Request Success
서울특별시 금천구 범안로  1203
[2018-08-04 16:57:07.057087] Url Request Success
서울특별시 강남구 선릉로158길 13-16
[2018-08-04 16:57:07.261540] Url Request Success
서울특별시 강남구 도산대로  426
[2018-08-04 16:57:07.397177] Url Request Success
서울특별시 서대문구 응암로  103
[2018-08-04 16:57:07.524837] Url Request Success
서울특별시 서초구 방배천로  21
[2018-08-04 16:57:07.653511] Url Request Success
서울특별시 마포구 어울마당로  94-13
[2018-08-04 16:57:07.7

In [21]:
with open('dish_place_tests.json', 'w', encoding="utf-8") as fp:
    json.dump(final_dict, fp, ensure_ascii=False, indent="\t")

In [91]:
import pandas as pd`

In [125]:
datas = DataFrame()
for info in final_dict.keys():
    data = pd.DataFrame.from_dict(final_dict[info])
    data = pd.DataFrame(data, columns=['place_name' , 'place_address', 'lat', 'lon'])
    data['category'] = info+"_food" 
    data['lat'] = data['lat'].astype(float)
    data['lon'] = data['lon'].astype(float)
    
    datas = datas.append(data)

In [126]:
datas

,place_name,place_address,lat,lon,category
0,FR푸드시스템,서울특별시 강서구 공항대로 318,37.557800,126.842915,korean_food
1,OK목장,서울특별시 송파구 백제고분로 547,NaN,NaN,korean_food
2,The만나,서울특별시 노원구 석계로 103,37.622842,127.060726,korean_food
3,㈜이리오어라(한정식),서울특별시 강남구 언주로153길 12,37.522956,127.032208,korean_food
4,가야옥,서울특별시 도봉구 도당로 2,37.658405,127.035329,korean_food
5,가족천하,서울특별시 영등포구 가마산로 330,37.498813,126.896346,korean_food
6,갈비둥지,서울특별시 노원구 덕릉로 692,37.659410,127.075689,korean_food
7,갈비사랑,서울특별시 서초구 강남대로 204,37.482488,127.036736,korean_food
8,갈비세상,서울특별시 구로구 경인로 281,37.498519,126.851831,korean_food
9,갈비예찬,서울특별시 도봉구 노해로 259,37.651681,127.036992,korean_food


In [127]:
datas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 414 entries, 0 to 100
Data columns (total 5 columns):
place_name       414 non-null object
place_address    414 non-null object
lat              407 non-null float64
lon              407 non-null float64
category         414 non-null object
dtypes: float64(2), object(3)
memory usage: 19.4+ KB


In [134]:
datas.to_csv("tests.csv",encoding = "utf-8")

TypeError: to_csv() got an unexpected keyword argument 'sheet_name'

In [135]:
test = pd.DataFrame.from_csv("tests.csv")

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  """Entry point for launching an IPython kernel.


In [136]:
test

,place_name,place_address,lat,lon,category
0,FR푸드시스템,서울특별시 강서구 공항대로 318,37.557800,126.842915,korean_food
1,OK목장,서울특별시 송파구 백제고분로 547,NaN,NaN,korean_food
2,The만나,서울특별시 노원구 석계로 103,37.622842,127.060726,korean_food
3,㈜이리오어라(한정식),서울특별시 강남구 언주로153길 12,37.522956,127.032208,korean_food
4,가야옥,서울특별시 도봉구 도당로 2,37.658405,127.035329,korean_food
5,가족천하,서울특별시 영등포구 가마산로 330,37.498813,126.896346,korean_food
6,갈비둥지,서울특별시 노원구 덕릉로 692,37.659410,127.075689,korean_food
7,갈비사랑,서울특별시 서초구 강남대로 204,37.482488,127.036736,korean_food
8,갈비세상,서울특별시 구로구 경인로 281,37.498519,126.851830,korean_food
9,갈비예찬,서울특별시 도봉구 노해로 259,37.651681,127.036992,korean_food


In [144]:
data = pd.DataFrame.from_dict(final_dict['korean'][0]['facility_detail_info'])

In [145]:
data

,facility_available_name,facility_info,facility_is_available
0,장애인 전용 주차구역,[총주차대수5],이용가능
1,주출입구 높이차이 제거,"[단차없음 또는 경사로 1/12이하 설치, 유효폭 1.2m 이상, 손잡이 설치]",이용가능


In [150]:
gettest = []
for test in final_dict.keys():
    for a in final_dict[test]:
        for b in a['facility_detail_info']:
            gettest.extend(set([info for info in b['facility_info']]))
gettest = set(gettest)
            

In [152]:
len(gettest)

99

In [153]:
gettest

{'가로조작반 설치',
 '경사로 1/12초과 설치',
 '관람석 면적0.9*1.3m 이상',
 '기울기 1/18~1/12이하',
 '기울기 1/18이하',
 '남,녀 구분 설치',
 '단차없음',
 '단차없음 또는 경사로 1/12이하 설치',
 '대변기 설치',
 '바닥단차 2cm 이하',
 '버튼점자 설치',
 '보/차도 구분',
 '보/차도 구분없음',
 '사용여부설비 설치',
 '상세정보 없음',
 '손잡이 미흡설치',
 '손잡이 설치',
 '수직손잡이 설치',
 '수평손잡이 설치',
 '승강기크기 1.6*1.35m 이상',
 '안내표지판 미설치',
 '안내표지판 설치',
 '양변기 설치',
 '유효폭 1.2m 미만',
 '유효폭 1.2m 이상',
 '음성안내 설치',
 '이동통로 1.2m 이상',
 '전용주차대수  1',
 '전용주차대수  2',
 '전용주차대수  3',
 '전용주차대수  4',
 '전용주차대수  5',
 '전용주차대수  6',
 '전용주차대수  8',
 '점멸등 설치',
 '점자블록 설치',
 '좌석설치',
 '총주차대수1',
 '총주차대수10',
 '총주차대수100',
 '총주차대수106',
 '총주차대수11',
 '총주차대수12',
 '총주차대수13',
 '총주차대수14',
 '총주차대수15',
 '총주차대수16',
 '총주차대수17',
 '총주차대수18',
 '총주차대수19',
 '총주차대수192',
 '총주차대수2',
 '총주차대수20',
 '총주차대수21',
 '총주차대수22',
 '총주차대수224',
 '총주차대수23',
 '총주차대수24',
 '총주차대수240',
 '총주차대수25',
 '총주차대수26',
 '총주차대수27',
 '총주차대수28',
 '총주차대수29',
 '총주차대수3',
 '총주차대수30',
 '총주차대수31',
 '총주차대수32',
 '총주차대수35',
 '총주차대수36',
 '총주차대수395',
 '총주차대수4',
 '총주차대수40',
 '총주차대수400',
 '총주차대수42',
 '총주차대수45',